# Trabájo Practico 2 - Métodos Computacionales

### Camila Cauzzo, Catalina Dolhare

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import time
import random

In [8]:
from derivada_w import fAndG as fAndG_w
from derivada_b import fAndG as fAndG_b

In [9]:
def abrirImagenesEscaladas( carpeta, escala=32 ):
    # abre todas las imagenes de la carpeta, y las escala de tal forma que midan (escala x escala)px
    # devuelve las imagenes aplanadas -> vectores de tamano escala^2 con valores entre 0 y 1
    imagenes = []

    for dirpath, dirnames, filenames in os.walk(carpeta):
        for file in filenames:
            img = Image.open( os.path.join(carpeta, file) )
            img = img.resize((escala, escala))
            img.convert('1')
            img = np.asarray(img)
            if len(img.shape)==3:
                img = img[:,:,0].reshape((escala**2 )) / 255
            else:
                img = img.reshape((escala**2 )) / 255
            
            imagenes.append( img )

    return imagenes

In [10]:
def balancear_datos():
    imagenes_train_normal = abrirImagenesEscaladas("chest_xray/train/NORMAL")
    imagenes_train_pneumonia = abrirImagenesEscaladas("chest_xray/train/PNEUMONIA")

    cantidad_imagenes = min(len(imagenes_train_normal), len(imagenes_train_pneumonia))
    imagenes_train_normal = imagenes_train_normal[:cantidad_imagenes]
    imagenes_train_pneumonia = imagenes_train_pneumonia[:cantidad_imagenes]

    return [imagenes_train_normal, imagenes_train_pneumonia]

### Ejercicio 1

Calculamos las derivadas parciales

In [39]:
def dw(i, w, b, d):
    t_0 = np.tanh((b + (w).dot(i)))
    t_1 = (((1 + t_0) / 2) - d)
    gradient = (((1 - (t_0 ** 2)) * t_1) * i)

    return gradient

In [40]:
def db(i, w, b, d):
    t_0 = np.tanh((b + (w).dot(i)))
    t_1 = (((1 + t_0) / 2) - d)
    gradient = ((1 - (t_0 ** 2)) * t_1)

    return gradient

In [41]:
def sum_dw(imagenes, w, b, d):
    gradient = np.zeros(len(w))
    for i in imagenes:
        gradient += dw(i, w, b, d)
    return gradient

In [42]:
def sum_db(imagenes, w, b, d):
    gradient = 0
    for i in imagenes:
        gradient += db(i, w, b, d)
    return gradient

In [13]:
def f(i, w, b):
    return (np.tanh(np.dot(w, i) + b)+1)/2

In [14]:
def L(imagenes, w, b):
    imagenes_normal = imagenes[0]
    imagenes_pneumonia = imagenes[1]

    sumL = 0

    for i in imagenes_normal:
        L = (f(i, w, b) - 0)**2
        sumL += L
    
    for i in imagenes_pneumonia:
        L = (f(i, w, b) - 1)**2
        sumL += L
        
    return sumL

## Ejercicio 2

Implementamos el metodo descenso por gradiente

In [46]:
def gradiente_descendiente(imagenes, alpha):
    imagenes_normales = imagenes[0]
    imagenes_pneumonia = imagenes[1]

    w = np.random.rand(32**2)
    b = np.random.rand(1)
    dw = np.zeros(w.shape)
    db = 0

    for i in range(1000):
        print("Iteracion: ", i, " L: ", L(imagenes, w, b))  
        for j in range(len(imagenes_normales)):
            dw = sum_dw(imagenes_normales, w, b, 0) + sum_dw(imagenes_pneumonia, w, b, 1)
            db = sum_db(imagenes_normales, w, b, 0) + sum_db(imagenes_pneumonia, w, b, 1)
        
        w = w - alpha * dw
        b = b - alpha * db

        print(L(imagenes, w, b))

    return w, b

In [ ]:
imagenes=balancear_datos()

In [47]:
w_opt, b_opt = gradiente_descendiente(imagenes, 0.0001)

Iteracion:  0  L:  [1349.]
[1349.]
Iteracion:  1  L:  [1349.]


KeyboardInterrupt: 